# HYPOTHESIS - Question Refers to Multiple Subjects


SimpleQuestions dataset example question is equivilant to (subject, relation, ?); therefore, for it to be answerable there must not be a duplicate (subject, relation, ?) tuple in the KG.

Within `HYPOTHESIS - Subject Name not in Question.ipynb` we suggest an algorithm for linking the subject alias to the question. 

Following we linked the subject alias referenced in the question, we can check if there exists other (subject, relation, ?).

In [3]:
import sys
sys.path.insert(0, '../../')
import pandas as pd
import random
from tqdm import tqdm_notebook
tqdm_notebook().pandas()
from lib.utils import FB2M_NAME_TABLE
from lib.utils import FB2M_KG_TABLE
from lib.utils import get_connection 
import lib.import_notebook

connection = get_connection()
cursor = connection.cursor()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

/usr/local/lib/python3.6/dist-packages/nbformat/current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)


In [4]:
from lib.simple_qa import load_simple_qa 

# Load development set because its a magnitude smaller than the training set.
df, = load_simple_qa(dev=True)
df[:5]

,subject,relation,object,question
0,0f3xg_,symbols/namesake/named_after,0cqt90,Who was the trump ocean club international hot...
1,07f3jg,people/person/place_of_birth,0565d,where was sasha vujačić born
2,031j8nn,music/release/region,07ssc,What is a region that dead combo was released in
3,0c1cyhd,film/director/film,0wxsz5y,What is a film directed by wiebke von carolsfeld?
4,0fvhc0g,music/release/region,0345h,what country was music for stock exchange rel...


## Step 1 - Link Question to Subject Alias

In [5]:
import importlib
module = importlib.import_module("notebooksSimple QA Numbers.HYPOTHESIS - Subject Name not in Question")
normalize = module.normalize

importing Jupyter notebook from ../../notebooks/Simple QA Numbers/HYPOTHESIS - Subject Name not in Question.ipynb


In [6]:
import pandas as pd
from lib.edit_distance import edit_token_distance
from IPython.display import display
from numpy import nan
from functools import partial

def edit_distance_link_alias(cursor, normalize, row, name_table=FB2M_NAME_TABLE):
    cursor.execute('SELECT alias FROM ' + name_table + ' WHERE mid=%s', (row['subject'],))
    aliases = [a[0] for a in cursor.fetchall()]
    normalized_aliases = [normalize(a) for a in aliases]
    for i, n_a in enumerate(normalized_aliases):
        if len(n_a.strip()) == 0:
            aliases.pop(i)
            normalized_aliases.pop(i)
    
    normalized_aliases = [tuple(a.split()) for a in normalized_aliases]
    if len(normalized_aliases) > 0:
        # NOTE: Given that there is no punctuation in the phrase then splitting based on white spaces works
        normalized_question = tuple(normalize(row['question']).split())
        distances = [edit_token_distance(n_a, normalized_question) for n_a in normalized_aliases]
        normalized_aliases_lengths = [sum(len(t) for t in n_a) for n_a in normalized_aliases]
        normalized_distances = [(normalized_aliases_lengths[i] - d[0]) / normalized_aliases_lengths[i]
                                for i, d in enumerate(distances)]
        
        if max(normalized_distances) >= 0.75:
            # NOTE: Pick the longest and least edit distance
            # TODO: Look at criteria for sorting
            subject_alias_index = max(range(len(normalized_distances)),
                                      key=lambda i: (normalized_distances[i], len(aliases[i])))
            subject_alias = aliases[subject_alias_index]
            return subject_alias
    return nan

# Create a column with the subject_name linked per example
df['subject_name'] = df.progress_apply(partial(edit_distance_link_alias, cursor, normalize), axis=1)
display(df[:5])

HBox(children=(IntProgress(value=0, max=10845), HTML(value='')))

,subject,relation,object,question,subject_name
0,0f3xg_,symbols/namesake/named_after,0cqt90,Who was the trump ocean club international hot...,trump ocean club international hotel and tower
1,07f3jg,people/person/place_of_birth,0565d,where was sasha vujačić born,sasha vujacic
2,031j8nn,music/release/region,07ssc,What is a region that dead combo was released in,dead combo
3,0c1cyhd,film/director/film,0wxsz5y,What is a film directed by wiebke von carolsfeld?,wiebke von carolsfeld
4,0fvhc0g,music/release/region,0345h,what country was music for stock exchange rel...,music for stock exchange


## Step 2 - Count Duplicates (subject, relation, ?)

In [5]:
from functools import lru_cache

@lru_cache(maxsize=10885)
def get_duplicate_mids(name_table, kg_table, subject_name, relation):
    cursor.execute("""
      SELECT subject_mid
      FROM {name_table}
      INNER JOIN {kg_table}
      ON subject_mid = mid
      WHERE alias = %s
      AND fb_two_kg.relation = %s""".format(name_table=name_table, kg_table=kg_table),
               (subject_name, relation))
    mids = set([mid for (mid,) in cursor.fetchall()])
    return mids

In [115]:
import statistics

import random

dups_data = []
n_dups = []

def count_duplicates(row, name_table=FB2M_NAME_TABLE, kg_table=FB2M_KG_TABLE):
    mids = get_duplicate_mids(name_table, kg_table, row['subject_name'], row['relation'])
    
    if len(mids) > 1:
        assert row['subject'] in mids
        dups_data.append({
            'Subject Name': row['subject_name'],
            'Relation': row['relation'],
            'Question': row['question'], 
            'Number of Duplicate MIDs': str(len(mids)),
            'True MID': row['subject'],
            'Duplicate MIDs': mids,
        })
    elif len(mids) == 0:
        # Unable to find any MIDs related to alias
        print('No MIDs for alias')
        print(row)

    n_dups.append(len(mids))

df[df.subject_name.notnull()].progress_apply(count_duplicates, axis=1)
display(pd.DataFrame(dups_data)[:50])

HBox(children=(IntProgress(value=0, max=10640), HTML(value='')))

,Duplicate MIDs,Number of Duplicate MIDs,Question,Relation,Subject Name,True MID
0,"{0b4pm3, 0gkrfmk, 0gmhlt, 0k3qrvh}",4,What is the language of the film bon voyage?,film/film/language,bon voyage,0gkrfmk
1,"{0ft53m8, 0dzb769, 0nl9mxz, 0rtntk, 0nlql9t, 0...",49,whats a version of the single titled star,music/single/versions,star,0nndnv_
2,"{02prsd4, 0zvwcn5, 0_hqxrc, 0_j2lh4, 0465qnd, ...",9,which recordings contains the composition aust...,music/composition/recordings,australia,0zvwcn5
3,"{09gbzy0, 0d5jxc, 0b1xf4, 08jcb1}",4,what country is ghost house from,film/film/country,ghost house,0b1xf4
4,"{0c03hf7, 01lftjn, 01hqqnq, 01m5vhv, 0gg5fdj, ...",20,what is the release type of the album wake?,music/album/release_type,wake,0fsn5x_
5,"{025wdlw, 02y49}",2,who is the chid of fritz leiber?,people/person/children,fritz leiber,02y49
6,"{04v7k_9, 04v7ky6, 04v7k_0, 04v7kzr, 04v7h85, ...",9,what is a book from the ethan frome series,book/book_edition/book,ethan frome,04v7kz6
7,"{03gt0jb, 0pc47fb, 0h566cd}",3,where is look from?,film/film/country,look,0pc47fb
8,"{0hqsrcv, 0hqrlp6}",2,what is the active ingredient moiety in fast f...,medicine/drug_formulation/active_ingredient_mo...,fast freeze,0hqrlp6
9,"{0177d57, 0247xcj, 014c2ft, 0dypv5t, 0fhg_23, ...",26,what is the name for meccas canonical version,music/recording/canonical_version,mecca,0fhg_23


## Analysis - Numbers

In [116]:
from IPython.display import display, Markdown

total_rows = df.shape[0]
n_rows = sum(df.subject_name.notnull())
expected_percision = sum(1.0 / n for n in n_dups if n != 0) / n_rows
one_duplicate = sum(1 for n in n_dups if n == 1)
zero_duplicate = sum(1 for n in n_dups if n == 0)


display(Markdown(
    '##### Filter by the `relation` and true `subject_mid`, then the duplicate distribution is:\n' + 
    '- Number of examples: %d\n' % n_rows + 
    '- Median number of duplicates: %f\n' % statistics.median(n_dups) + 
    '- Max number of duplicates: %f\n' % max(n_dups) + 
    '- Mean number of duplicates: %f\n' % statistics.mean(n_dups) + 
    '- Stdev number of duplicates: %f\n' % statistics.stdev(n_dups) + 
    '- `subject_name`s with 0 duplicate: %f [%d of %d]\n' % (zero_duplicate / n_rows, zero_duplicate, n_rows) +  
    '- `subject_name`s with 1 duplicate: %f [%d of %d]\n' % (one_duplicate / n_rows, one_duplicate, n_rows) +  
    '- Expected percision with 100%% recall and random guessing: %f\n' % expected_percision
))

##### Filter by the `relation` and true `subject_mid`, then the duplicate distribution is:
- Number of examples: 10640
- Median number of duplicates: 1.000000
- Max number of duplicates: 2077.000000
- Mean number of duplicates: 11.373778
- Stdev number of duplicates: 73.552866
- `subject_name`s with 0 duplicate: 0.000000 [0 of 10640]
- `subject_name`s with 1 duplicate: 0.826692 [8796 of 10640]
- Expected percision with 100% recall and random guessing: 0.872740


## Step 3 - Better than random guessing

Given {s_0, s_1, ...}, r, and {o_0, o_1, ...} the subject with the most atomic facts (s, r, o) in KG is most likely to get picked. The white paper states that their sampling strategy is a function f(r); therefore, if a particular subject has more atomic facts with relation r then is has a higher chance of getting picked during sampling.

In [8]:
import random 

correct = 0

def expected_accuracy(row, name_table=FB2M_NAME_TABLE, kg_table=FB2M_KG_TABLE):
    global correct
    mids = get_duplicate_mids(name_table, kg_table, row['subject_name'], row['relation'])
    mids = list(mids)
    if len(mids) > 1:
        scores = []
        for mid in mids:
            cursor.execute("""
                SELECT count(*)
                FROM {kg_table}
                WHERE subject_mid = %s
                AND relation = %s
            """.format(kg_table=kg_table), (mid, row['relation']))
            # Score is the number of facts associated with (s, r, ?)
            scores.append(cursor.fetchall()[0][0])
        max_score = max(scores)
        max_mids = [mids[i] for i, s in enumerate(scores) if s == max_score]
        if row['subject'] in max_mids:
            correct += 1 / len(max_mids)
    elif len(mids) == 1 and mids[0] == row['subject']:
        correct += 1
      
df[df.subject_name.notnull()].progress_apply(expected_accuracy, axis=1)
n_rows = sum(df.subject_name.notnull())
print('Better than Random Guessing Accuracy: %f [%d of %d]' % ((correct/ n_rows), correct, n_rows))

HBox(children=(IntProgress(value=0, max=10640), HTML(value='')))


Better than Random Guessing Accuracy: 0.892452 [9495 of 10640]


## Analysis - Empirical Examples of Duplicates

Here we randomly sample the duplicates our algorithm found to assist in empirical analysis and verification of the algorithms correctness.

The goal is to print out a random sample of duplicates and the facts associated with each duplicate. Empirically we verify that it's impossible for a human to tell apart the duplicate from the True answer.

In [156]:
from collections import defaultdict
import pprint

pp = pprint.PrettyPrinter(indent=2)
MAX_SAMPLE_DUPLICATE_MIDS = 5

def get_replaced_by(mid):
    # Check if the subject has been replaced; Otherwise, return the latter MID.
    cursor.execute("""
        SELECT object
        FROM fb
        WHERE subject = %s
        AND relation = '<http://rdf.freebase.com/ns/dataworld.gardening_hint.replaced_by>'
        """, (mid,))
    rows = [r[0] for r in cursor.fetchall()]
    assert len(rows) < 2
    if len(rows) == 1:
        return rows[0]
    else:
        return mid

def get_aliases(subject):
    # Get aliases for a subject. If there are no aliases, it returns the subject.
    if 'http://rdf.freebase.com/ns/m.' in subject:
        mid = subject
        cursor.execute("""
            SELECT object
            FROM fb
            WHERE subject = %s
            AND object LIKE '%%@en'
            AND relation IN ('<http://rdf.freebase.com/ns/type.object.name>',
                             '<http://rdf.freebase.com/ns/common.topic.alias>',
                             '<http://www.w3.org/2000/01/rdf-schema#label>',
                             '<http://rdf.freebase.com/ns/medicine.drug_formulation.brand_names>')
            """, (mid,))
        aliases = tuple(set([r[0] for r in cursor.fetchall()]))
        if len(aliases) == 0:
            return subject
        return aliases
    else:
        return subject

for i, row in tqdm_notebook(enumerate(random.sample(dups_data, 25))):
    print('\n\n[%s] Question: %s' % (i, row['Question']))
    print('Subject Name:', row['Subject Name'])
    print('Relation:', row['Relation'])
    print('Number of Duplicates:', len(row['Duplicate MIDs']))

    # Sample + Format duplicate MIDS
    duplicate_mids = list(row['Duplicate MIDs'])
    duplicate_mids = ['<http://rdf.freebase.com/ns/m.' + mid + '>' for mid in duplicate_mids]
    # NOTE: Some MIDs have been replaced; therefore, its important for us to follow that link
    duplicate_mids = set([get_replaced_by(m) for m in duplicate_mids])
    print('Replaced Duplicates (Same MID via dataworld.gardening_hint.replaced_by):', 
          len(row['Duplicate MIDs']) - len(duplicate_mids))
    sample_size = min(len(duplicate_mids), MAX_SAMPLE_DUPLICATE_MIDS)
    duplicate_mids = random.sample(duplicate_mids, sample_size)
    
    # Get aliases for the duplicate MIDS to ensure that the duplicate MIDs are referenced in the question
    print_aliases = [(mid, get_aliases(mid)) for mid in duplicate_mids]
    print('ALIASES:')
    pp.pprint(print_aliases)
    
    # GOAL: Facts should be pruned to only be facts useful for comparing the subjects
    # GOAL: Print facts in a human readable way to allow for quick empirical analysis
    # Get all facts for all duplicates mids
    cursor.execute("""
        SELECT subject, relation, object
        FROM fb
        WHERE subject = ANY(%s)
    """, (duplicate_mids,))
    facts = cursor.fetchall()
    
    # Get english names for Freebase nodes to make it human readable
    facts = [(s, r, get_aliases(o)) for (s, r, o) in facts]
    
    # Filter facts every subject has in common
    common_facts = defaultdict(set)
    for s, r, o in facts:
        common_facts[(r, o)].add(s)
    facts = [(s, r, o) for (s, r, o) in facts if len(common_facts[(r, o)]) < sample_size]

    # Filter relations that every duplicate subject does not address
    # Those relations are less helpful at finding differences amoung subjects
    relation_to_subjects = defaultdict(set)
    for subject, relation, object_ in facts:
        relation_to_subjects[relation].add(subject)
    facts = [(s, r, o) for (s, r, o) in facts if len(relation_to_subjects[r]) >= sample_size]
    
    # Group together facts and pretty print
    print_facts = defaultdict(lambda: defaultdict(lambda: set()))
    for s, r, o in facts:
        # NOTE: Freebase nodes are not helpful
        if 'http://rdf.freebase.com/ns/m.' in o or 'http://rdf.freebase.com/ns/g.' in o:
            continue
        
        # NOTE: Descriptions are long and not helpful
        # TODO: Not sure how to use to type.object.key but lots of facts are around that relation
        if 'common.topic.description' in r or 'key' in r:
            continue

        r = r.replace('<http://rdf.freebase.com/', '')
        r = r.replace('>', '')
        s = s.replace('<http://rdf.freebase.com/ns/m.', '')
        s = s.replace('>', '')
        if isinstance(o, (set, list)):
            print_facts[r][s].update(o)
        else:
            print_facts[r][s].add(o)

    # Print
    # Remove defaultdict to print prettier
    print_facts = dict(print_facts)
    for r, values in print_facts.items():
        print_facts[r] = dict(values)
    
    print('COMPARABLE FACTS:')
    pp.pprint(print_facts)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))



[0] Question: What is the language of the film bon voyage?
Subject Name: bon voyage
Relation: film/film/language
Number of Duplicates: 4
Replaced Duplicates (Same MID via dataworld.gardening_hint.replaced_by): 0
ALIASES:
[ ('<http://rdf.freebase.com/ns/m.0k3qrvh>', ('"Bon Voyage"@en',)),
  ('<http://rdf.freebase.com/ns/m.0gmhlt>', ('"Bon Voyage"@en',)),
  ('<http://rdf.freebase.com/ns/m.0gkrfmk>', ('"Bon Voyage"@en',)),
  ('<http://rdf.freebase.com/ns/m.0b4pm3>', ('"Bon Voyage"@en',))]
COMPARABLE FACTS:
{ '<http://www.w3.org/2000/01/rdf-schema#label': { '0b4pm3': { '"Bon '
                                                               'Voyage"@ca',
                                                               '"Bon '
                                                               'Voyage"@fi',
                                                               '"Bon '
                                                               'Voyage"@fr',
                                             

COMPARABLE FACTS:
{ 'ns/common.topic.notable_types': { '02prsd4': { ( '"Composition"@en',
                                                    '"Song"@en')},
                                     '0465qnd': { ( '"Composition"@en',
                                                    '"Song"@en')},
                                     '0_62f1p': { ( '"Composition"@en',
                                                    '"Song"@en')},
                                     '0_hqxrc': { ( '"Composition"@en',
                                                    '"Song"@en')},
                                     '0kpg4sd': {('"TV theme song"@en',)}},
  'ns/common.topic.topic_equivalent_webpage': { '02prsd4': { '<http://en.wikipedia.org/wiki/Australia_(The_Shins_song)>',
                                                             '<http://en.wikipedia.org/wiki/index.html?curid=9773455>'},
                                                '0465qnd': { '<http://en.wikipedia.org/wiki/Australia_(Gyro

COMPARABLE FACTS:
{ 'ns/common.topic.topic_equivalent_webpage': { '01lftjn': { '<http://en.wikipedia.org/wiki/Wake_(Emmet_Swimming_album)>',
                                                             '<http://en.wikipedia.org/wiki/index.html?curid=40727880>',
                                                             '<http://musicbrainz.org/release-group/10ae3bda-5ffa-3eb9-a799-9521dac3df4f>',
                                                             '<http://musicbrainz.org/release-group/acafbe7d-76a3-3219-8b18-b7bd899eb6ed>'},
                                                '01p060l': { '<http://en.wikipedia.org/wiki/Wake_(Trio_T%F6yke%E4t_album)>',
                                                             '<http://en.wikipedia.org/wiki/index.html?curid=2840406>',
                                                             '<http://musicbrainz.org/release-group/fca2e93c-2959-3be0-b8b5-ba098bfcfb63>'},
                                                '03jmqrc': { '<http://m

COMPARABLE FACTS:
{ '<http://www.w3.org/2000/01/rdf-schema#label': { '03gt0jb': { '"Look"@en',
                                                                '"Look"@es',
                                                                '"Look"@fr',
                                                                '"룩"@ko'},
                                                   '0h566cd': { '"Dyut meng '
                                                                'gam"@ro',
                                                                '"Kuralsız '
                                                                'Hayat"@tr',
                                                                '"La Vie sans '
                                                                'principe"@fr',
                                                                '"Life Without '
                                                                'Principle"@de',
                                               

...

COMPARABLE FACTS:
{ '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type': { '0488_j0': { '<http://rdf.freebase.com/ns/base.type_ontology.agent>',
                                                                     '<http://rdf.freebase.com/ns/location.citytown>'},
                                                        '049_nd8': { '<http://rdf.freebase.com/ns/base.type_ontology.agent>',
                                                                     '<http://rdf.freebase.com/ns/location.citytown>'},
                                                        '0qymh': { '<http://rdf.freebase.com/ns/base.wikipedia_infobox.settlement>',
                                                                   '<http://rdf.freebase.com/ns/location.census_designated_place>',
                                                                   '<http://rdf.freebase.com/ns/location.hud_county_place>',
                                                                   '<http://rdf.freebase.com/ns/loca

COMPARABLE FACTS:
{ '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type': { '08s4zc': { '<http://rdf.freebase.com/ns/film.actor>',
                                                                    '<http://rdf.freebase.com/ns/influence.influence_node>',
                                                                    '<http://rdf.freebase.com/ns/people.deceased_person>',
                                                                    '<http://rdf.freebase.com/ns/tv.tv_actor>'},
                                                        '090rbh1': { '<http://rdf.freebase.com/ns/film.film_crewmember>'},
                                                        '0gc3lhh': { '<http://rdf.freebase.com/ns/film.actor>'},
                                                        '0gc63b2': { '<http://rdf.freebase.com/ns/film.actor>',
                                                                     '<http://rdf.freebase.com/ns/film.person_or_entity_appearing_in_film>',
                     

COMPARABLE FACTS:
{ '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type': { '0114q4': { '<http://rdf.freebase.com/ns/base.type_ontology.agent>',
                                                                    '<http://rdf.freebase.com/ns/base.wikipedia_infobox.settlement>',
                                                                    '<http://rdf.freebase.com/ns/location.citytown>'},
                                                        '0480bx_': { '<http://rdf.freebase.com/ns/base.type_ontology.agent>',
                                                                     '<http://rdf.freebase.com/ns/location.citytown>'},
                                                        '04b2tsy': { '<http://rdf.freebase.com/ns/base.type_ontology.agent>',
                                                                     '<http://rdf.freebase.com/ns/base.wikipedia_infobox.settlement>',
                                                                     '<http://rdf.freebase.com/ns

COMPARABLE FACTS:
{ '<http://www.w3.org/2000/01/rdf-schema#label': { '072192': { '"A Place in '
                                                               'the Sun"@ca',
                                                               '"A Place in '
                                                               'the Sun"@en',
                                                               '"A Place in '
                                                               'the Sun"@es',
                                                               '"A Place in '
                                                               'the Sun"@ms',
                                                               '"A Place in '
                                                               'the Sun"@nl',
                                                               '"Egy hely a '
                                                               'nap alatt"@hu',
                                            

COMPARABLE FACTS:
{}


[23] Question: The recording star is an adaptation of which song?
Subject Name: star
Relation: music/recording/canonical_version
Number of Duplicates: 158
Replaced Duplicates (Same MID via dataworld.gardening_hint.replaced_by): 1
ALIASES:
[ ('<http://rdf.freebase.com/ns/m.0tdbc5>', ('"Star"@en',)),
  ('<http://rdf.freebase.com/ns/m.018_w84>', ('"Star"@en',)),
  ('<http://rdf.freebase.com/ns/m.0smgv62>', ('"Star"@en',)),
  ('<http://rdf.freebase.com/ns/m.0dpx3b9>', ('"Star"@en',)),
  ('<http://rdf.freebase.com/ns/m.0tb8_q>', ('"Star"@en',))]
COMPARABLE FACTS:
{ 'ns/common.topic.topic_equivalent_webpage': { '018_w84': { '<http://musicbrainz.org/recording/7bba0a8b-e06a-403c-9b03-62f2e00df4b2>'},
                                                '0dpx3b9': { '<http://musicbrainz.org/recording/77a2f4e8-7b78-4238-9250-df6cb2f840f5>'},
                                                '0smgv62': { '<http://musicbrainz.org/recording/a0c36857-e4a5-489b-acdc-9f896ceb511d>'},
 